1. Split the data in two samples, a training sample and a hold-out sample (make sure to be clear about how you split the data set).


First draft model which includes all the parameters.
I will drop the variables that are not significant(p<0.05) in the revised model 

In [148]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from scipy import stats
data = pd.read_csv('https://raw.githubusercontent.com/ormarketing/b2b/master/data.csv')
data.head(10)

,ID,Close Date,Created Date,New Logo,Type,Stage,Billing Plan,ACV,Discount Amount,Amount,Net Amount
0,1,5/26/17,5/4/17,1,Direct - Cloud Product,Closed Lost,3 Years,431370,,225000,225000
1,2,12/29/16,11/29/16,1,Partner - OnPremise Product,Closed Won - Paid,3 Years,22050,80850,147000,66150
2,3,12/29/16,12/9/16,1,Partner - Cloud Product,Closed Won - Paid,3 Years,32750,43200,96000,52800
3,4,9/9/16,4/20/16,1,Direct - Cloud Product,Closed Won - Paid,4 Years,36694,44800,128000,83200
4,5,9/22/16,9/1/16,1,Partner - OnPremise Product,Closed Won - Paid,3 Years,18700,9900,66000,56100
5,6,12/6/16,9/12/16,1,Partner - OnPremise Product,Closed Won - Paid,3 Years,22400,28800,96000,67200
6,7,9/26/16,6/24/16,1,Partner - OnPremise Product,Closed Won - Paid,3 Years,38000,111000,225000,114000
7,8,7/14/16,7/12/16,1,Direct - OnPremise Product,Closed Won - Paid,3 Years,16500,16500,66000,49500
8,9,9/1/16,7/13/16,1,Partner - OnPremise Product,Closed Won - Paid,3 Years,17600,13200,66000,52800
9,10,12/30/16,8/9/16,1,Partner - OnPremise Product,Closed Won - Paid,3 Years,22050,80850,147000,66150


In [149]:
# Remove extra spaces from column names
data.columns = data.columns.str.strip()

# Extract year from Billing Plan and convert to float
data['Billing Plan'] = data['Billing Plan'].str.extract('(\d+)').astype(float)

# Convert relevant columns to numeric, coercing errors
for col in ['ACV', 'Discount Amount', 'Amount', 'Net Amount']:
    data[col] = pd.to_numeric(data[col].str.strip(), errors='coerce')

# Convert date columns to datetime
data['Close Date'] = pd.to_datetime(data['Close Date'])

# Handle missing values (for simplicity, we'll fill NaNs with 0; you may choose a different strategy)
data.fillna(0, inplace=True)

# Add quarter column based on Close Date
data['Quarter'] = data['Close Date'].dt.quarter

# One-hot encode Type and New Logo
data = pd.get_dummies(data, columns=['Type', 'New Logo'], drop_first=True)

# Define the target variable
data['won'] = data['Stage'].apply(lambda x: 1 if 'Won' in x else 0)

# Define predictors
predictors = ['Discount Amount', 'Amount', 'Billing Plan', 'Quarter'] + \
             [col for col in data.columns if col.startswith('Type_') or col.startswith('New Logo_')]

# Ensure all predictor columns are numeric and have no NaNs
data[predictors] = data[predictors].apply(pd.to_numeric, errors='coerce')
data = data.dropna(subset=predictors)

# Check for NaN values (should be none)
print(data[predictors].isnull().sum())

Discount Amount                     0
Amount                              0
Billing Plan                        0
Quarter                             0
Type_Direct - OnPremise Product     0
Type_Partner - Cloud Product        0
Type_Partner - OnPremise Product    0
New Logo_1                          0
dtype: int64


In [150]:
data.head()

,ID,Close Date,Created Date,Stage,Billing Plan,ACV,Discount Amount,Amount,Net Amount,Quarter,Type_Direct - OnPremise Product,Type_Partner - Cloud Product,Type_Partner - OnPremise Product,New Logo_1,won
0,1,2017-05-26,5/4/17,Closed Lost,3.0,431370.0,0.0,225000.0,225000.0,2,0,0,0,1,0
1,2,2016-12-29,11/29/16,Closed Won - Paid,3.0,22050.0,80850.0,147000.0,66150.0,4,0,0,1,1,1
2,3,2016-12-29,12/9/16,Closed Won - Paid,3.0,32750.0,43200.0,96000.0,52800.0,4,0,1,0,1,1
3,4,2016-09-09,4/20/16,Closed Won - Paid,4.0,36694.0,44800.0,128000.0,83200.0,3,0,0,0,1,1
4,5,2016-09-22,9/1/16,Closed Won - Paid,3.0,18700.0,9900.0,66000.0,56100.0,3,0,0,1,1,1


In [151]:
# Split the data into training and holdout sets
train_data, holdout_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data['won'])

# Prepare the training data
X_train = train_data[predictors]
y_train = train_data['won']

# Fit the logistic regression model
logit_model = LogisticRegression(max_iter=1000)
logit_model.fit(X_train, y_train)

# Get the coefficients and intercept
coefficients = logit_model.coef_[0]
intercept = logit_model.intercept_[0]

# Predict probabilities for the training set
train_preds = logit_model.predict_proba(X_train)[:, 1]

# Calculate log loss for the training set
train_log_loss = log_loss(y_train, train_preds)

print("Coefficients:", coefficients)
print("Intercept:", intercept)
print("Training Log Loss:", train_log_loss)

Coefficients: [ 4.55352788e-05 -4.67582504e-06  4.02427722e-01 -4.45170359e-01
 -1.11625527e-01  6.63924812e-04 -8.09342400e-02  1.02185688e-01]
Intercept: -0.21157646320311396
Training Log Loss: 0.5076472285837417


In [152]:
# Predict probabilities for the holdout set
holdout_preds = logit_model.predict_proba(X_holdout)[:, 1]

# Calculate log loss for the holdout set
holdout_log_loss = log_loss(y_holdout, holdout_preds)

print("Holdout Log Loss:", holdout_log_loss)

Holdout Log Loss: 0.4769874750944362


In [153]:
def calculate_p_values(model, X, y):
    X_with_intercept = np.hstack([np.ones((X.shape[0], 1)), X])
    preds = model.predict_proba(X)[:, 1]
    ll = y * np.log(preds) + (1 - y) * np.log(1 - preds)
    ll = -ll.sum()
    hessian = np.dot((preds * (1 - preds) * X_with_intercept.T), X_with_intercept)
    se = np.sqrt(np.diag(np.linalg.inv(hessian)))
    z = np.concatenate((np.array([model.intercept_[0]]), model.coef_[0])) / se
    p_values = [2 * (1 - stats.norm.cdf(np.abs(i))) for i in z]
    return p_values

p_values = calculate_p_values(logit_model, X_train, y_train)
coef_pval_df = pd.DataFrame({
    'Predictor': ['Intercept'] + predictors,
    'Coefficient': [intercept] + list(coefficients),
    'P-value': p_values
})
print(coef_pval_df)


                          Predictor  Coefficient       P-value
0                         Intercept    -0.211576  5.822116e-01
1                   Discount Amount     0.000046  8.393286e-14
2                            Amount    -0.000005  1.323552e-03
3                      Billing Plan     0.402428  9.030241e-10
4                           Quarter    -0.445170  5.080136e-11
5   Type_Direct - OnPremise Product    -0.111626  7.602303e-01
6      Type_Partner - Cloud Product     0.000664  9.985898e-01
7  Type_Partner - OnPremise Product    -0.080934  8.152080e-01
8                        New Logo_1     0.102186  5.284006e-01


In [154]:
# Define the negative revenue function
def revenue(p):
    """
    This is the negative revenue function.
    p is a vector containing values for all predictors.
    The first two elements are the list price and the discount.
    """
    x = p
    prob_win = np.exp(intercept + np.dot(coefficients, x)) / (1 + np.exp(intercept + np.dot(coefficients, x)))
    return -prob_win * (p[0] - p[1])

# Initial condition: mean values of all predictors
init = [np.mean(train_data[col]) for col in predictors]

# Optimize the price and discount
sol = minimize(revenue, init, method='BFGS', tol=0.001)

# Print the success status and the optimal values
print(sol.success)
print(f"Optimal Price: {sol.x[0]}")
print(f"Optimal Discount: {sol.x[1]}")

# Compare with mean values of 'Amount' and 'Discount Amount'
print(f"un-optimized Mean Amount: {np.mean(data['Amount'])}")
print(f"un-optimized Mean Discount: {np.mean(data['Discount Amount'])}")


True
Optimal Price: 13525.525691902067
Optimal Discount: 78577.39856968565
un-optimized Mean Amount: 80728.09235880399
un-optimized Mean Discount: 14447.42392026578


In [155]:
# Filter out variables with p-values greater than 0.05
significant_predictors = coef_pval_df[coef_pval_df['P-value'] <= 0.05]['Predictor'].tolist()
if 'Intercept' in significant_predictors:
    significant_predictors.remove('Intercept')  # Remove intercept from the list of predictors if present

print("\nSignificant Predictors (P-value <= 0.05):")
print(significant_predictors)


Significant Predictors (P-value <= 0.05):
['Discount Amount', 'Amount', 'Billing Plan', 'Quarter']


In [156]:

# Refit the model with only significant predictors
X_train_significant = train_data[significant_predictors]
logit_model_significant = LogisticRegression(max_iter=1000)
logit_model_significant.fit(X_train_significant, y_train)

# Prepare the holdout data with significant predictors
X_holdout_significant = holdout_data[significant_predictors]
y_holdout = holdout_data['won']

# Predict probabilities for the holdout set
holdout_preds_significant = logit_model_significant.predict_proba(X_holdout_significant)[:, 1]

# Calculate log loss for the holdout set with significant predictors
holdout_log_loss_significant = log_loss(y_holdout, holdout_preds_significant)

print("Holdout Log Loss with Significant Predictors:", holdout_log_loss_significant)

Holdout Log Loss with Significant Predictors: 0.4834871053197552


I built a logit model with quarter from the close date, billing plan, discount amount, amount and type as independent variables and stage as dependent variable.

Later, I ran a t test to find the p value of each  independent variables to see which is significant parameters. As a result, 'Discount Amount', 'Amount', 'Billing Plan', 'Quarter' are significant parameters.
For the train and holdout data accuracy, the log loss are similar: 0.5076 and 0.4770 respectively. Howevwer, the log loss didn't improve after taking out insignificant variables, given holdout Log Loss with Significant Predictors is 0.4835.

Here is the result of the mean of Optimal Price, Optimal Discount, un-optimized Mean Amount, and un-optimized Mean Discount for the whole dataset. 

Optimal Price: 13525.525691902067
Optimal Discount: 78577.39856968565
un-optimized Mean Amount: 80728.09235880399
un-optimized Mean Discount: 14447.42392026578


Prompt:
1. the whole question 
2. the formula from the reading
3. reference to the code/frormula from class to get price and discount : 
Define the negative revenue function
def revenue(p):
    """
    This is the negative revenue function.
    p is a vector containing values for all predictors.
    The first two elements are the list price and the discount.
    """
    x = p
    prob_win = np.exp(intercept + np.dot(coefficients, x)) / (1 + np.exp(intercept + np.dot(coefficients, x)))
    return -prob_win * (p[0] - p[1])
4. get the log loss for handout data
5. calculate p value with each variable's name and get the varibles that has a p value less than 0.05